<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/CHATBOT_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask openai colab-env -q

  Preparing metadata (setup.py) ... done


In [2]:
!pip install pyngrok -q  # Install pyngrok

In [ ]:
!pip install --upgrade openai -q

In [ ]:
!curl -sSL https://ngrok-agent.s3.amazonaws.com/ngrok.asc \
	| sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null \
	&& echo "deb https://ngrok-agent.s3.amazonaws.com buster main" \
	| sudo tee /etc/apt/sources.list.d/ngrok.list \
	&& sudo apt update \
	&& sudo apt install ngrok

FINAL_PROD

In [ ]:
from IPython import get_ipython
from IPython.display import display
import os
from dotenv import load_dotenv
from flask import Flask, render_template_string, request
import openai
import threading
from IPython.display import HTML, display
import socket
import time
import traceback
from pyngrok import ngrok, conf
import colab_env

# Load environment variables from .env file (optional)
load_dotenv()

# Access API keys from environment variables
openai.api_key = os.getenv("OPENAI_API_KEY")
if openai.api_key is None:
    raise ValueError("OPENAI_API_KEY environment variable not set.")

# --- Flask App Setup ---
app = Flask(__name__)

conversation_history = []  # Initialize conversation history

def get_response(user_input):
    """Gets a response from the OpenAI GPT-3.5-turbo API."""
    try:
        conversation_history.append({"role": "user", "content": user_input})

        #print("Sending request to OpenAI API...")
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=conversation_history
        )
        print("Received response from OpenAI API:", response)

        assistant_response = response.choices[0].message.content.strip()
        conversation_history.append({"role": "assistant", "content": assistant_response})
        return assistant_response

    except openai.OpenAIError as e:
        print(f"OpenAI API Error: {e}")
        if isinstance(e, openai.AuthenticationError):
            return "Authentication error. Please check your API key."
        elif isinstance(e, openai.RateLimitError):
            return "Rate limit exceeded. Please try again later."
        elif isinstance(e, openai.APIConnectionError):
            return "Could not connect to OpenAI. Check your network."
        else:
            return "An error occurred. Please try again later."

    except Exception as e:
        traceback.print_exc()
        return "An error occurred."

@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        user_input = request.form["user_input"]
        status_message = "Sending request to OpenAI API..."
        response = get_response(user_input)
        return render_template_string('''
            <!DOCTYPE html>
            <html>
            <head>
              <title>Simple Chatbot</title>
              <style>
                #chat {
                  border: 1px solid #ccc;
                  padding: 10px;
                  height: 300px;
                  overflow-y: scroll;
                }
                .message {  /* Common styles for both user and assistant messages */
                  padding: 5px 10px;
                  margin-bottom: 5px;
                  border-radius: 5px;
                }
                .user-message {
                  background-color: #eee;
                  text-align: left;  /* Align user messages to the left */
                }
                .assistant-message {
                  background-color: #ccf;
                  text-align: left;  /* Align assistant messages to the left */
                }
              </style>
            </head>
            <body>
              <h1>Simple Chatbot - POC by Frank Morales</h1>
              <form method="POST">
                <input type="text" name="user_input" placeholder="Enter your message" autofocus>
                <button type="submit">Send</button>
              </form>
              <div id="chat">
                <p>{{ status_message }}</p>
                {% for message in conversation_history %}
                  {% if message.role == 'user' %}
                    <div class="message user-message">  <b>You:</b> {{ message.content }}
                    </div>
                  {% elif message.role == 'assistant' %}
                    <div class="message assistant-message">  <b>Chatbot:</b> {{ message.content }}
                    </div>
                  {% endif %}
                {% endfor %}
              </div>
              <script>
                var chatDiv = document.getElementById('chat');
                chatDiv.scrollTop = chatDiv.scrollHeight;
              </script>
            </body>
            </html>
        ''', conversation_history=conversation_history, status_message=status_message)
    return render_template_string('''
            <!DOCTYPE html>
            <html>
            <head>
              <title>Simple Chatbot</title>
              <style>
                #chat {
                  border: 1px solid #ccc;
                  padding: 10px;
                  height: 300px;
                  overflow-y: scroll;
                }
                .message {  /* Common styles for both user and assistant messages */
                  padding: 5px 10px;
                  margin-bottom: 5px;
                  border-radius: 5px;
                }
                .user-message {
                  background-color: #eee;
                  text-align: left;  /* Align user messages to the left */
                }
                .assistant-message {
                  background-color: #ccf;
                  text-align: left;  /* Align assistant messages to the left */
                }
              </style>
            </head>
            <body>
              <h1>Simple Chatbot - POC by Frank Morales</h1>
              <form method="POST">
                <input type="text" name="user_input" placeholder="Enter your message">
                <button type="submit">Send</button>
              </form>
              <div id="chat"></div>
            </body>
            </html>
        ''')


stop_event = threading.Event()  # Create a threading event

def run_app(app):
    """Starts the Flask app and exposes it using ngrok."""
    port = 0

    def get_free_port():
        """Finds a free port."""
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.bind(('', 0))
            return s.getsockname()[1]

    port = get_free_port()

    def flask_thread():
        """Runs the Flask app in a separate thread."""
        with app.app_context():  # Use app context
            try:
                app.run(host="0.0.0.0", port=port, debug=True, use_reloader=False)
            except:
                pass  # Ignore exceptions during shutdown

    thread = threading.Thread(target=flask_thread)
    thread.start()

    time.sleep(1)

    # Configure ngrok
    conf.get_default().auth_token = os.getenv("NGROK_TOKEN")
    if conf.get_default().auth_token is None:
        raise ValueError("NGROK_TOKEN environment variable not set.")

    # Terminate existing ngrok tunnels
    active_tunnels = ngrok.get_tunnels()
    for tunnel in active_tunnels:
        ngrok.disconnect(tunnel.public_url)

    # Create a new ngrok tunnel
    public_url = ngrok.connect(port, bind_tls=True).public_url
    print(f" * Running on {public_url}")
    display(HTML(f'<a href="{public_url}">Open the Chatbot</a>'))

    # Wait for the stop event
    stop_event.wait()

    # Clean up ngrok tunnel (optional)
    ngrok.disconnect(public_url)


if __name__ == "__main__":
    try:
        run_app(app)
    finally:
        stop_event.set()  # Signal the Flask app to shut down

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:32861
 * Running on http://172.28.0.2:32861
INFO:werkzeug:Press CTRL+C to quit


 * Running on https://6837-35-196-190-246.ngrok-free.app


INFO:werkzeug:127.0.0.1 - - [14/Jan/2025 17:36:39] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Jan/2025 17:36:39] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [14/Jan/2025 17:36:44] "POST / HTTP/1.1" 200 -


Received response from OpenAI API: ChatCompletion(id='chatcmpl-Apf8Gz8SF99Gf0GM264C6JrnrdjVW', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of Cuba is Havana.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1736876204, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=8, prompt_tokens=17, total_tokens=25, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
